In [1]:
from datasets import load_dataset
import pandas as pd
from datasets import Dataset, DatasetDict, load_from_disk
from sklearn.model_selection import train_test_split

In [2]:
# Load the dataset from a JSON file
qa_dataset = pd.read_json('../../data/english_to_urdu_translations/openai_translations_v2.json')
qa_dataset = qa_dataset.drop_duplicates(subset='english_question', keep='first')
display(qa_dataset.head())

,urdu_question,urdu_answer,english_question,english_answer
0,پاکستان میں کس قسم کی حکومت ہے؟,پاکستان ایک وفاقی جمہوریہ ہے جسے اسلامی جمہوری...,What type of government does Pakistan have?,Pakistan is a Federal Republic known as the Is...
1,کیا نئی ریاستیں یا علاقے پاکستان کا حصہ بن سکت...,جی ہاں، مجلس شوریٰ (پارلیمنٹ) بذریعہ قانون وفا...,Can new states or areas become part of Pakistan?,"Yes, the Majlis-e-Shoora (Parliament) may admi..."
2,پاکستان کا مقررہ مذہب کیا ہے؟,اسلام ریاست کا مذہب ہوگا۔,What is the designated religion of Pakistan?,Islam shall be the State religion of Pakistan.
3,دئے گئے متن کے مطابق دستور کا مستقل حصہ کیا بن...,قرارداد مقاصد میں بیان کردہ اصول اور احکام کو ...,What has been made a substantive part of the C...,The principles and provisions set out in the O...
4,ریاست کی استحصال کے بارے میں کیا ذمہ داری ہے؟,ریاست استحصال کی تمام اقسام کے خاتمے کو یقینی ...,What is the State's responsibility regarding e...,The State shall ensure the elimination of all ...


In [3]:
contexts_dataset = pd.read_csv('../../data/document_chunks/article_wise_chunks_eng_urdu_processed.csv')

display(contexts_dataset.head())
print(len(contexts_dataset))

,English_Context_Chunks,Urdu_Context_Chunks,urdu_cleaned,Context_Index
0,The Republic and its territories\n11.\t(1)\tPa...,ﺍ۔ ﻣﻤﻠﮑﺖ ﭘﺎﮐﺴﺘﺎﻥ ﺍﯾﮏ ﻭﻓﺎﻗﯽ ﺟﻤﮩﻮﺭﯾﮧ ﮨﻮﮔﯽ ﺟﺲ ﮐﺎ...,۱۔ مملکت پاکستان ایک وفاقی جمہوریہ ہوگی جس کا ...,0
1,Islam to be State religion\n\nIslam shall be t...,ﺍﺳﻼﻡ ﻣﻠﮑﺘﯽ ﻣﺬﮨﺐ ﮨﻮﮔﺎ۔,اسلام ریاست کا مذہب ہوگا۔,1
2,The Objectives\tResolution to form\tpart...,ﺍﺳﻼﻡ ﭘﺎﮐﺴﺘﺎﻥ ﮐﺎﻣﻤﻠﮑﺘﯽ ﻣﺬﮨﺐ ﮨﻮﮔﺎ۔\n\n۲۲ ﺍﻟﻒ۔ ﺿﻤ...,اسلام پاکستان کا مملکتی مذہب ہوگا۔\n\n۲۲ الف۔ ...,2
3,Elimination of exploitation\n\nThe State shall...,ﺍﺳﺘﺤﺼﺎﻝ ﮐﺎ ﺧﺎﺗﻤﮧ۔ ﻣ۳۔ﻤﻠﮑﺖ ﺍﺳﺘﺤﺼﺎﻝ ﮐﯽ ﺗﻤﺎﻡ ﺍﻗﺴﺎ...,استحصال کا خاتمہ\n\nریاست استحصال کی تمام اقسا...,3
4,Right of individuals to be dealt with in accor...,ﻗﺎﻧﺍﻮﻓﺮﻥﺍﺩﻭﮐﻏﺎﯿﺮﺣﮦﻖﮐﮐﮯﮧ ﺍﻥ ﺳﮯ۴۔۴ﮨﺮﺷﮩﺮﯼ ﮐﺎﺧﻮﺍﮨﮑ...,افراد کا حق کہ ان کے ساتھ قانون کے مطابق سلوک ...,4


312


In [4]:
enlgish_qa_dataset = pd.read_csv('../../data/augmented_datasets/qa_dataset_eng_v1.csv')
enlgish_qa_dataset = enlgish_qa_dataset.drop_duplicates(subset='Question', keep='first')
enlgish_qa_dataset

,Question,Answer,Answer Chunk Index
0,What type of government does Pakistan have?,Pakistan is a Federal Republic known as the Is...,0
1,Can new states or areas become part of Pakistan?,"Yes, the Majlis-e-Shoora (Parliament) may admi...",0
2,What is the designated religion of Pakistan?,Islam shall be the State religion of Pakistan.,1
3,What has been made a substantive part of the C...,The principles and provisions set out in the O...,2
4,What is the State's responsibility regarding e...,The State shall ensure the elimination of all ...,3
...,...,...,...
616,Who is responsible for handling accounts that ...,The Auditor-General is responsible for handlin...,309
617,What powers does the Auditor-General have rega...,The Auditor-General has the same powers and fu...,309
618,Will existing taxes continue to be collected a...,"Yes, all taxes and fees levied under any law i...",310
619,When was the Proclamation of Emergency mention...,The Proclamation of Emergency was originally i...,311


## Merging Datasets

In [5]:
merged_context_eng = pd.merge(enlgish_qa_dataset, contexts_dataset, left_on='Answer Chunk Index', right_on='Context_Index', how='inner')
display(merged_context_eng.head())

,Question,Answer,Answer Chunk Index,English_Context_Chunks,Urdu_Context_Chunks,urdu_cleaned,Context_Index
0,What type of government does Pakistan have?,Pakistan is a Federal Republic known as the Is...,0,The Republic and its territories\n11.\t(1)\tPa...,ﺍ۔ ﻣﻤﻠﮑﺖ ﭘﺎﮐﺴﺘﺎﻥ ﺍﯾﮏ ﻭﻓﺎﻗﯽ ﺟﻤﮩﻮﺭﯾﮧ ﮨﻮﮔﯽ ﺟﺲ ﮐﺎ...,۱۔ مملکت پاکستان ایک وفاقی جمہوریہ ہوگی جس کا ...,0
1,Can new states or areas become part of Pakistan?,"Yes, the Majlis-e-Shoora (Parliament) may admi...",0,The Republic and its territories\n11.\t(1)\tPa...,ﺍ۔ ﻣﻤﻠﮑﺖ ﭘﺎﮐﺴﺘﺎﻥ ﺍﯾﮏ ﻭﻓﺎﻗﯽ ﺟﻤﮩﻮﺭﯾﮧ ﮨﻮﮔﯽ ﺟﺲ ﮐﺎ...,۱۔ مملکت پاکستان ایک وفاقی جمہوریہ ہوگی جس کا ...,0
2,What is the designated religion of Pakistan?,Islam shall be the State religion of Pakistan.,1,Islam to be State religion\n\nIslam shall be t...,ﺍﺳﻼﻡ ﻣﻠﮑﺘﯽ ﻣﺬﮨﺐ ﮨﻮﮔﺎ۔,اسلام ریاست کا مذہب ہوگا۔,1
3,What has been made a substantive part of the C...,The principles and provisions set out in the O...,2,The Objectives\tResolution to form\tpart...,ﺍﺳﻼﻡ ﭘﺎﮐﺴﺘﺎﻥ ﮐﺎﻣﻤﻠﮑﺘﯽ ﻣﺬﮨﺐ ﮨﻮﮔﺎ۔\n\n۲۲ ﺍﻟﻒ۔ ﺿﻤ...,اسلام پاکستان کا مملکتی مذہب ہوگا۔\n\n۲۲ الف۔ ...,2
4,What is the State's responsibility regarding e...,The State shall ensure the elimination of all ...,3,Elimination of exploitation\n\nThe State shall...,ﺍﺳﺘﺤﺼﺎﻝ ﮐﺎ ﺧﺎﺗﻤﮧ۔ ﻣ۳۔ﻤﻠﮑﺖ ﺍﺳﺘﺤﺼﺎﻝ ﮐﯽ ﺗﻤﺎﻡ ﺍﻗﺴﺎ...,استحصال کا خاتمہ\n\nریاست استحصال کی تمام اقسا...,3


In [6]:
full_dataset = pd.merge(merged_context_eng, qa_dataset, left_on='Question', right_on='english_question', how='inner')
display(full_dataset.head())

,Question,Answer,Answer Chunk Index,English_Context_Chunks,Urdu_Context_Chunks,urdu_cleaned,Context_Index,urdu_question,urdu_answer,english_question,english_answer
0,What type of government does Pakistan have?,Pakistan is a Federal Republic known as the Is...,0,The Republic and its territories\n11.\t(1)\tPa...,ﺍ۔ ﻣﻤﻠﮑﺖ ﭘﺎﮐﺴﺘﺎﻥ ﺍﯾﮏ ﻭﻓﺎﻗﯽ ﺟﻤﮩﻮﺭﯾﮧ ﮨﻮﮔﯽ ﺟﺲ ﮐﺎ...,۱۔ مملکت پاکستان ایک وفاقی جمہوریہ ہوگی جس کا ...,0,پاکستان میں کس قسم کی حکومت ہے؟,پاکستان ایک وفاقی جمہوریہ ہے جسے اسلامی جمہوری...,What type of government does Pakistan have?,Pakistan is a Federal Republic known as the Is...
1,Can new states or areas become part of Pakistan?,"Yes, the Majlis-e-Shoora (Parliament) may admi...",0,The Republic and its territories\n11.\t(1)\tPa...,ﺍ۔ ﻣﻤﻠﮑﺖ ﭘﺎﮐﺴﺘﺎﻥ ﺍﯾﮏ ﻭﻓﺎﻗﯽ ﺟﻤﮩﻮﺭﯾﮧ ﮨﻮﮔﯽ ﺟﺲ ﮐﺎ...,۱۔ مملکت پاکستان ایک وفاقی جمہوریہ ہوگی جس کا ...,0,کیا نئی ریاستیں یا علاقے پاکستان کا حصہ بن سکت...,جی ہاں، مجلس شوریٰ (پارلیمنٹ) بذریعہ قانون وفا...,Can new states or areas become part of Pakistan?,"Yes, the Majlis-e-Shoora (Parliament) may admi..."
2,What is the designated religion of Pakistan?,Islam shall be the State religion of Pakistan.,1,Islam to be State religion\n\nIslam shall be t...,ﺍﺳﻼﻡ ﻣﻠﮑﺘﯽ ﻣﺬﮨﺐ ﮨﻮﮔﺎ۔,اسلام ریاست کا مذہب ہوگا۔,1,پاکستان کا مقررہ مذہب کیا ہے؟,اسلام ریاست کا مذہب ہوگا۔,What is the designated religion of Pakistan?,Islam shall be the State religion of Pakistan.
3,What has been made a substantive part of the C...,The principles and provisions set out in the O...,2,The Objectives\tResolution to form\tpart...,ﺍﺳﻼﻡ ﭘﺎﮐﺴﺘﺎﻥ ﮐﺎﻣﻤﻠﮑﺘﯽ ﻣﺬﮨﺐ ﮨﻮﮔﺎ۔\n\n۲۲ ﺍﻟﻒ۔ ﺿﻤ...,اسلام پاکستان کا مملکتی مذہب ہوگا۔\n\n۲۲ الف۔ ...,2,دئے گئے متن کے مطابق دستور کا مستقل حصہ کیا بن...,قرارداد مقاصد میں بیان کردہ اصول اور احکام کو ...,What has been made a substantive part of the C...,The principles and provisions set out in the O...
4,What is the State's responsibility regarding e...,The State shall ensure the elimination of all ...,3,Elimination of exploitation\n\nThe State shall...,ﺍﺳﺘﺤﺼﺎﻝ ﮐﺎ ﺧﺎﺗﻤﮧ۔ ﻣ۳۔ﻤﻠﮑﺖ ﺍﺳﺘﺤﺼﺎﻝ ﮐﯽ ﺗﻤﺎﻡ ﺍﻗﺴﺎ...,استحصال کا خاتمہ\n\nریاست استحصال کی تمام اقسا...,3,ریاست کی استحصال کے بارے میں کیا ذمہ داری ہے؟,ریاست استحصال کی تمام اقسام کے خاتمے کو یقینی ...,What is the State's responsibility regarding e...,The State shall ensure the elimination of all ...


In [7]:
# Step 1: Select only the required columns
processed_dataset = full_dataset[['english_question', 'urdu_question', 'English_Context_Chunks', 'urdu_cleaned', 'english_answer', 'urdu_answer', 'Context_Index']]

processed_dataset = processed_dataset.rename(columns={
    'english_question': 'question_eng',
    'urdu_question': 'question_urdu',
    'English_Context_Chunks': 'context_eng',
    'urdu_cleaned': 'context_urdu',
    'english_answer': 'answer_eng',
    'urdu_answer': 'answer_urdu',
    'Context_Index': 'context_index'
})

display(processed_dataset.head())

,question_eng,question_urdu,context_eng,context_urdu,answer_eng,answer_urdu,context_index
0,What type of government does Pakistan have?,پاکستان میں کس قسم کی حکومت ہے؟,The Republic and its territories\n11.\t(1)\tPa...,۱۔ مملکت پاکستان ایک وفاقی جمہوریہ ہوگی جس کا ...,Pakistan is a Federal Republic known as the Is...,پاکستان ایک وفاقی جمہوریہ ہے جسے اسلامی جمہوری...,0
1,Can new states or areas become part of Pakistan?,کیا نئی ریاستیں یا علاقے پاکستان کا حصہ بن سکت...,The Republic and its territories\n11.\t(1)\tPa...,۱۔ مملکت پاکستان ایک وفاقی جمہوریہ ہوگی جس کا ...,"Yes, the Majlis-e-Shoora (Parliament) may admi...",جی ہاں، مجلس شوریٰ (پارلیمنٹ) بذریعہ قانون وفا...,0
2,What is the designated religion of Pakistan?,پاکستان کا مقررہ مذہب کیا ہے؟,Islam to be State religion\n\nIslam shall be t...,اسلام ریاست کا مذہب ہوگا۔,Islam shall be the State religion of Pakistan.,اسلام ریاست کا مذہب ہوگا۔,1
3,What has been made a substantive part of the C...,دئے گئے متن کے مطابق دستور کا مستقل حصہ کیا بن...,The Objectives\tResolution to form\tpart...,اسلام پاکستان کا مملکتی مذہب ہوگا۔\n\n۲۲ الف۔ ...,The principles and provisions set out in the O...,قرارداد مقاصد میں بیان کردہ اصول اور احکام کو ...,2
4,What is the State's responsibility regarding e...,ریاست کی استحصال کے بارے میں کیا ذمہ داری ہے؟,Elimination of exploitation\n\nThe State shall...,استحصال کا خاتمہ\n\nریاست استحصال کی تمام اقسا...,The State shall ensure the elimination of all ...,ریاست استحصال کی تمام اقسام کے خاتمے کو یقینی ...,3


## Convert to dataset format

In [8]:
train_df, val_df = train_test_split(processed_dataset, test_size=0.2, random_state=25)

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

In [9]:
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['question_eng', 'question_urdu', 'context_eng', 'context_urdu', 'answer_eng', 'answer_urdu', 'context_index', '__index_level_0__'],
        num_rows: 495
    })
    validation: Dataset({
        features: ['question_eng', 'question_urdu', 'context_eng', 'context_urdu', 'answer_eng', 'answer_urdu', 'context_index', '__index_level_0__'],
        num_rows: 124
    })
})


In [10]:
dataset_dict.save_to_disk("../../data/complete_dataset/")

Saving the dataset (0/1 shards):   0%|          | 0/495 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/124 [00:00<?, ? examples/s]

### Loading instructions

In [11]:
dataset_dict = load_from_disk("../../data/complete_dataset/")

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['question_eng', 'question_urdu', 'context_eng', 'context_urdu', 'answer_eng', 'answer_urdu', 'context_index', '__index_level_0__'],
        num_rows: 495
    })
    validation: Dataset({
        features: ['question_eng', 'question_urdu', 'context_eng', 'context_urdu', 'answer_eng', 'answer_urdu', 'context_index', '__index_level_0__'],
        num_rows: 124
    })
})
